This wee script combines the unWISE files for the VHzQs. 
Starting with 'just' the original 424 VHzQs and then added
the "Halloween" 2018 Oct objects from arXiv:1810.11924-27v1. 


In [1]:
import numpy as np
from astropy.io import ascii
from astropy.io import fits

In [2]:
fourhundered_w1_hdu = fits.open("VHzQ_w1-unwise_source_catalog.fits")  
fourhundered_w2_hdu = fits.open("VHzQ_w1-unwise_source_catalog.fits")  
halloween_w1_hdu    = fits.open("halloween_quasars-w1.fits")
halloween_w2_hdu    = fits.open("halloween_quasars-w2.fits")

In [3]:
fourhundered_w1_hdu

[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x10c00ac18>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x10c02c320>]

In [4]:
fourhundered_w1 = fourhundered_w1_hdu[1].data
fourhundered_w1 = fourhundered_w1_hdu[1].data
halloween_w1    = halloween_w1_hdu[1].data
halloween_w2    = halloween_w2_hdu[1].data

In [5]:
#fourhundered_w1.columns
#halloween_w1.columns     ##   Doesn't have OBJID, NM

In [6]:
print(type(fourhundered_w1['RA']), len(fourhundered_w1['RA']))
print(type(halloween_w1['RA'])   , len(halloween_w1['RA']))

<class 'numpy.ndarray'> 322
<class 'numpy.ndarray'> 33


In [7]:
## Combining the W1 arrays
unWISE_W1_RA    = np.concatenate((fourhundered_w1['RA'], halloween_w1['RA']), axis=0)
unWISE_W1_DEC   = np.concatenate((fourhundered_w1['DEC'], halloween_w1['DEC']), axis=0)
unWISE_W1_FLUX  = np.concatenate((fourhundered_w1['FLUX'], halloween_w1['FLUX']), axis=0)
unWISE_W1_DFLUX = np.concatenate((fourhundered_w1['DFlux'], halloween_w1['DFlux']), axis=0)


In [8]:
print(len(unWISE_W1_RA))

355


In [9]:
def fluxtomag(nmgy):
    return -2.5 * (np.log10(np.maximum(1e-3, nmgy)) - 9.)

def dfluxtodmag(nmgy, dnmgy):
    return np.abs(-2.5 * (np.log10(1. + dnmgy / nmgy)))


In [12]:
unWISE_W1_mag    = fluxtomag(unWISE_W1_FLUX)
unWISE_W1_magerr = dfluxtodmag(unWISE_W1_FLUX, unWISE_W1_DFLUX)
unWISE_W1_SNR    = unWISE_W1_FLUX/unWISE_W1_DFLUX


In [ ]:
for x, y, z in zip(unWISE_W1_mag, unWISE_W1_magerr,unWISE_W1_SNR ):
    print(x, y, z)

In [13]:
data_out = [unWISE_W1_RA, unWISE_W1_DEC, unWISE_W1_mag, unWISE_W1_magerr, unWISE_W1_SNR]

In [15]:
type(data_out)

list

In [21]:
ascii.write(data_out, 'unWISE_W1_temp.dat', names=['ra', 'dec', ' w1mpro', 'w1sigmpro', 'w1snr'], overwrite=True)